# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846820888168                   -0.70    4.5
  2   -7.852307954849       -2.26       -1.53    1.0
  3   -7.852646202562       -3.47       -2.52    3.2
  4   -7.852646676948       -6.32       -3.35    2.2
  5   -7.852646685959       -8.05       -4.74    1.5
  6   -7.852646686725       -9.12       -5.24    3.5
  7   -7.852646686730      -11.35       -6.02    1.0
  8   -7.852646686730      -12.54       -7.45    2.2
  9   -7.852646686730   +  -14.75       -7.59    3.2
 10   -7.852646686730      -14.45       -8.94    2.0
 11   -7.852646686730   +  -14.75       -9.42    2.0
 12   -7.852646686730      -14.57      -10.72    3.0
 13   -7.852646686730   +  -14.45      -11.04    1.2
 14   -7.852646686730      -14.75      -11.64    2.0
 15   -7.852646686730      -15.05      -12.34    2.5


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846850176298                   -0.70           4.8
  2   -7.852554401971       -2.24       -1.63   0.80    2.0
  3   -7.852640082715       -4.07       -2.72   0.80    1.0
  4   -7.852646522072       -5.19       -3.38   0.80    2.0
  5   -7.852646683707       -6.79       -4.31   0.80    1.5
  6   -7.852646686595       -8.54       -4.91   0.80    1.5
  7   -7.852646686721       -9.90       -5.76   0.80    1.5
  8   -7.852646686730      -11.07       -6.42   0.80    2.5
  9   -7.852646686730      -12.90       -7.20   0.80    1.2
 10   -7.852646686730      -13.88       -7.80   0.80    2.0
 11   -7.852646686730   +  -15.05       -9.00   0.80    1.2
 12   -7.852646686730   +  -14.75       -9.34   0.80    2.8
 13   -7.852646686730   +  -15.05      -10.53   0.80    1.0
 14   -7.852646686730      -15.05      -11.18   0.80    2.5
 15   -7.852646686730   +    -Inf      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.382230e+01     3.088731e+00
 * time: 0.47106313705444336
     1     9.909403e-01     2.009202e+00
 * time: 0.6850690841674805
     2    -1.631625e+00     2.340765e+00
 * time: 0.7087600231170654
     3    -3.811220e+00     1.953074e+00
 * time: 0.7426400184631348
     4    -5.156808e+00     1.628624e+00
 * time: 0.7767541408538818
     5    -6.823026e+00     1.215498e+00
 * time: 0.8106801509857178
     6    -6.876862e+00     1.664093e+00
 * time: 0.8343541622161865
     7    -7.535643e+00     1.556286e+00
 * time: 0.8582501411437988
     8    -7.550807e+00     2.605009e+00
 * time: 0.8819961547851562
     9    -7.627954e+00     2.420604e+00
 * time: 0.9061160087585449
    10    -7.650049e+00     2.193387e+00
 * time: 0.9403212070465088
    11    -7.656949e+00     2.104877e+00
 * time: 0.9746661186218262
    12    -7.662145e+00     1.970000e+00
 * time: 1.0090911388397217
    13    -7.720648e+00     3.404679e-01
 * time: 1.054423093

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846819299804                   -0.70    4.5
  2   -7.852319456340       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686711                   -2.55
  2   -7.852646686730      -10.73       -5.99
  3   -7.852646686730   +    -Inf      -12.62


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 4.0096467090361775e-13
|ρ_newton - ρ_scfv| = 4.372682917405269e-12
|ρ_newton - ρ_dm|   = 1.5669978087659768e-9
